In [ ]:
## Importando a biblioteca
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
## Carregando o modelo treinado
model=load_model('modeldi.h5')

## Carregando o encoder e o scaler
with open('onehot_encoder_smoking.pkl','rb') as file:
    onehot_encoder_smoking=pickle.load(file)

with open('label_encoder_genderdi.pkl','rb') as file:
    label_encoder_genderdi=pickle.load(file)

with open('scalerdi.pkl','rb') as file:
    scalerdi=pickle.load(file)